In [ ]:
#Bibliotheken importieren:

import requests
from bs4 import BeautifulSoup as soup
import lxml.etree as ET
import unicodedata
import pandas as pd

# Datenaufbereitung

In [ ]:
# Funktion mit Schleife: 

def dnb_sru(searchterm, dataform): 
    
    base_url = "https://services.dnb.de/sru/dnb"
    parameter = {'version' : '1.1' , 
                 'operation' : 'searchRetrieve' , 
                 'query' : searchterm, 
                 'recordSchema' : dataform,
                 'maximumRecords': '100'} 
    
    r1 = requests.get(base_url, params = parameter)  
    response = soup(r1.content, features="xml")
    records = response.find_all('record', {'type':'Bibliographic'})
    
    results = response.find('numberOfRecords').text  
    print("Gefundene Treffer:", results)
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            parameter.update({'startRecord': i})
            r = requests.get(base_url, params=parameter)
            xml = soup(r.content, features="xml")
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

    
    return records  

In [ ]:
result = dnb_sru("Bibliothekswesen and location=onlinefree", "MARC21-xml")
print(len(result))

In [ ]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = ET.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # title
    title = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        title = title[0].text
    except:
        title = "unknown"
        
    
    # author
    author = xml.xpath("marc:datafield[@tag = '100']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        author = author[0].text
    except:
        author = "unknown"
       
       
 
    #date
    date1 = xml.xpath("marc:datafield[@tag = '264']/marc:subfield[@code = 'c']", namespaces=ns)
    date2 = xml.xpath("marc:datafield[@tag = '502']/marc:subfield[@code = 'd']", namespaces=ns)
    date3 = xml.xpath("marc:datafield[@tag = '502']/marc:subfield[@code = 'a']", namespaces=ns)
    if date1:
        date = date1[0].text
    elif date2:
        date = date2[0].text
    elif date3:
        date = date3[0].text
    else:
        date = "none"
        
 
    
    meta_dict = {"idn":idn, "title":title, "author":author, "date":date}
    
    return meta_dict

In [ ]:
output = [parse_record(record) for record in result]
df = pd.DataFrame(output)
df

In [ ]:
df_dates = df.loc[df['date'] != "none"]
df_dates


In [ ]:
print(df_dates.date)#.to_string(index=False))

In [ ]:
#df_dates['date'] = df['date'].str.strip("[]")
df_dates['date'] = df_dates['date'].str.replace('\D', '', regex=True)
df_dates

In [ ]:
#print(df_dates.date)#.to_string(index=False))
df_dates.query("date.str.len() > 4")

In [ ]:
df_dates['date'] = df_dates['date'].str[:4]
df_dates

In [ ]:
#df_dates.query("date.str.len() > 4")
print(df_dates.date.to_string(index=False))

In [ ]:
df_dates.to_csv("BiblioCon.csv", encoding="utf-8")

## Advanced

In [ ]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = ET.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # title
    title = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        title = title[0].text
    except:
        title = "unknown"
        
    
    # author
    author = xml.xpath("marc:datafield[@tag = '100']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        author = author[0].text
    except:
        author = "unknown"
       
       
 
    #date
    date = xml.xpath("marc:datafield[@tag = '264']/marc:subfield[@code = 'c']", namespaces=ns)
    date2 = xml.xpath("marc:datafield[@tag = '502']/marc:subfield[@code = 'd']", namespaces=ns)
    date3 = xml.xpath("marc:datafield[@tag = '502']/marc:subfield[@code = 'a']", namespaces=ns)
    date4 = xml.xpath("marc:datafield[@tag = '773']/marc:subfield[@code = 'g']", namespaces=ns)
    test = ""
    if date:
        date = date[0].text
    elif date2:
        date = date2[0].text
    elif date3:
        date = date3[0].text
    elif date4:
        for item in date4:
            test+= item.text + ", "
        date = test
    else:
        date = "none"
        
 
    
    meta_dict = {"idn":idn, "title":title, "author":author, "date":date}
    
    return meta_dict


In [ ]:
output = [parse_record(record) for record in result]
df = pd.DataFrame(output)
df

In [ ]:
df.query("date.str.len() > 4")